In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [80]:
# Importing Answers-Weather_Prediction_Pleasant_Weather dataset
answers = pd.read_csv(r"C:\Users\isaac\Documents\CareerFoundry\3. Machine Learning with Python\Basics of ML for Analysis\04 Analysis\Data Sets\01 Raw Data\Answers-Weather_Prediction_Pleasant_Weather.csv")

# Importing Dataset-weather-prediction-dataset-processed dataset
weather = pd.read_csv(r"C:\Users\isaac\Documents\CareerFoundry\3. Machine Learning with Python\Basics of ML for Analysis\04 Analysis\Data Sets\01 Raw Data\Dataset-weather-prediction-dataset-processed.csv")

# Verifying imports
answers.head(), weather.head()

(       DATE  BASEL_pleasant_weather  BELGRADE_pleasant_weather  \
 0  19600101                       0                          0   
 1  19600102                       0                          0   
 2  19600103                       0                          0   
 3  19600104                       0                          0   
 4  19600105                       0                          0   
 
    BUDAPEST_pleasant_weather  DEBILT_pleasant_weather  \
 0                          0                        0   
 1                          0                        0   
 2                          0                        0   
 3                          0                        0   
 4                          0                        0   
 
    DUSSELDORF_pleasant_weather  HEATHROW_pleasant_weather  \
 0                            0                          0   
 1                            0                          0   
 2                            0                          0  

In [3]:
weather.shape

(22950, 170)

In [82]:
# Verify exported files
path_data = r'C:\Users\isaac\Documents\CareerFoundry\3. Machine Learning with Python\Real-World Applications of ML\02 Data'

weather_path = os.path.join(path_data, 'weather_cleaned.csv')
answers_path = os.path.join(path_data, 'answers_cleaned.csv')

# Load the CSVs
weather_exported = pd.read_csv(weather_path)
answers_exported = pd.read_csv(answers_path)

# Check columns
print("Weather columns:", weather_exported.columns)
print("Answers columns:", answers_exported.columns)


Weather columns: Index(['BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure',
       'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine',
       'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max',
       'BELGRADE_cloud_cover',
       ...
       'STOCKHOLM_temp_max', 'VALENTIA_cloud_cover', 'VALENTIA_humidity',
       'VALENTIA_pressure', 'VALENTIA_global_radiation',
       'VALENTIA_precipitation', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max'],
      dtype='object', length=135)
Answers columns: Index(['BASEL_pleasant_weather', 'BELGRADE_pleasant_weather',
       'BUDAPEST_pleasant_weather', 'DEBILT_pleasant_weather',
       'DUSSELDORF_pleasant_weather', 'HEATHROW_pleasant_weather',
       'KASSEL_pleasant_weather', 'LJUBLJANA_pleasant_weather',
       'MAASTRICHT_pleasant_weather', 'MADRID_pleasant_weather',
       'MUNCHENB_pleasant_weather', 'OSLO_pleasant_weather',
       'SONNBLICK_pleasant_weather', 'STOCKHOLM_ple

## Data Wrangling

In [5]:
weather = weather.drop([
    # Gdansk
    'GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation',
    'GDANSK_snow_depth', 'GDANSK_temp_mean', 'GDANSK_temp_min', 'GDANSK_temp_max',
    
    # Rome
    'ROMA_cloud_cover', 'ROMA_wind_speed', 'ROMA_humidity', 'ROMA_pressure',
    'ROMA_sunshine', 'ROMA_temp_mean',
    
    # Tours
    'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_pressure', 'TOURS_global_radiation',
    'TOURS_precipitation', 'TOURS_temp_mean', 'TOURS_temp_min', 'TOURS_temp_max'
], axis=1)


In [6]:
weather.shape

(22950, 149)

### Removing Observation Types Missing in Most Stations

Two observation types are problematic:

- wind_speed (only 9 stations had data)
- now_depth (only 6 stations had data)

Since they are missing for most other stations, drop any columns that end with _wind_speed or _snow_depth:

In [8]:
# Identify columns to drop
columns_to_drop = weather.filter(regex='(_wind_speed|_snow_depth)$').columns
print(columns_to_drop)  # Just to verify

# Drop those columns
weather = weather.drop(columns=columns_to_drop)

Index(['BASEL_wind_speed', 'BASEL_snow_depth', 'DEBILT_wind_speed',
       'DUSSELDORF_wind_speed', 'DUSSELDORF_snow_depth', 'HEATHROW_snow_depth',
       'KASSEL_wind_speed', 'LJUBLJANA_wind_speed', 'MAASTRICHT_wind_speed',
       'MADRID_wind_speed', 'MUNCHENB_snow_depth', 'OSLO_wind_speed',
       'OSLO_snow_depth', 'SONNBLICK_wind_speed', 'VALENTIA_snow_depth'],
      dtype='object')


In [9]:
weather.shape

(22950, 134)

In [72]:
# Exporting cleaned data set
clean_data_folder = r"C:\Users\isaac\Documents\CareerFoundry\3. Machine Learning with Python\Real-World Applications of ML\02 Data"

# Export 'weather' to the cleaned dataset in that folder
weather.to_csv(os.path.join(clean_data_folder, 'weather_cleaned.csv'), index=False)

# Export 'answers' to the cleaned dataset in that folder
answers.to_csv(os.path.join(clean_data_folder, 'answers_cleaned.csv'), index=False)

print("Export completed!")

Export completed!


### Fill 3 Individual Observations from Nearby Stations

Three specific columns were missing:
- KASSEL_cloud_cover (we copied from DUSSELDORF_cloud_cover)
- STOCKHOLM_humidity (we copied from OSLO_humidity)
- MUNCHENB_pressure (we copied from BASEL_pressure)

In [11]:
# locating columns (to figure out where to insert)
weather.columns.get_loc('HEATHROW_temp_max')  
# returns an index so we know where to insert Kassel’s data

# Insert new columns (copying from nearest station)
weather.insert(56, 'KASSEL_cloud_cover', weather['DUSSELDORF_cloud_cover'])
weather.insert(119, 'STOCKHOLM_humidity', weather['OSLO_humidity'])
weather.insert(94, 'MUNCHENB_pressure', weather['BASEL_pressure'])

weather.columns
# Verify they are inserted as expected.


Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_humidity',
       'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
       'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min',
       ...
       'STOCKHOLM_temp_max', 'VALENTIA_cloud_cover', 'VALENTIA_humidity',
       'VALENTIA_pressure', 'VALENTIA_global_radiation',
       'VALENTIA_precipitation', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max'],
      dtype='object', length=137)

In [68]:
# Verify the columns
weather.columns
# Verify they are inserted as expected.
# Example output of weather.columns
# Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_humidity',
#        'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
#        'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min',
#        'STOCKHOLM_temp_max', 'VALENTIA_cloud_cover', 'VALENTIA_humidity',
#        'VALENTIA_pressure', 'VALENTIA_global_radiation',
#        'VALENTIA_precipitation', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
#        'VALENTIA_temp_min', 'VALENTIA_temp_max'],
#       dtype='object', length=137)

# Do not drop DATE and MONTH from Observations
# weather.drop(['DATE', 'MONTH'], axis=1, inplace=True)

# Do not drop DATE from answers
# answers.drop(columns='DATE', inplace=True)

# Export cleaned data sets:
clean_data_folder = r"C:\Users\isaac\Documents\CareerFoundry\3. Machine Learning with Python\Real-World Applications of ML\02 Data"

# Export 'weather' to the cleaned dataset in that folder
weather.to_csv(os.path.join(clean_data_folder, 'weather_cleaned.csv'), index=False)

# Export 'answers' to the cleaned dataset in that folder
answers.to_csv(os.path.join(clean_data_folder, 'answers_cleaned.csv'), index=False)

### Drop DATE and MONTH from Observations; Drop DATE from answers

In [13]:
weather.drop(['DATE', 'MONTH'], axis=1, inplace=True)
weather.shape
# (22950, 135)  <-- matches the required shape for X


(22950, 135)

In [14]:
answers.drop(columns='DATE', inplace=True)
answers.shape
# (22950, 15)


(22950, 15)

### Exporting cleaned data set:

In [16]:
# Folder path
clean_data_folder = r"C:\Users\isaac\Documents\CareerFoundry\3. Machine Learning with Python\Real-World Applications of ML\02 Data"

# Export 'weather' to the cleaned dataset in that folder
weather.to_csv(os.path.join(clean_data_folder, 'weather_cleaned.csv'),index=False)

# Export 'answers' to the cleaned dataset in that folder
answers.to_csv(os.path.join(clean_data_folder, 'answers_cleaned.csv'),index=False)

---

## Reshape the Data for Modeling

In [19]:
# Load your data
# Path to your cleaned dataset
path = r"C:\Users\isaac\Documents\CareerFoundry\3. Machine Learning with Python\Real-World Applications of ML\02 Data"

X = pd.read_csv(os.path.join(path, 'weather_cleaned.csv'))
y = pd.read_csv(os.path.join(path, 'answers_cleaned.csv'))

In [20]:
print(X.shape)  # should be (22950, 135)
print(y.shape)  # should be (22950, 15)

(22950, 135)
(22950, 15)


### Reshape 𝑋

In [22]:
# Convert X and y to NumPy arrays
X = X.values  # or np.array(X)
y = y.values  # or np.array(y)

# Reshape from (22950, 135) --> (22950, 15, 9)
X = X.reshape(-1, 15, 9)
print(X.shape)  # should be (22950, 15, 9)
print(y.shape)  # should remain (22950, 15)


(22950, 15, 9)
(22950, 15)


**Data Shapes Explanation**  
- X is now (22950, 15, 9): 22950 samples, each with 15 time steps, each time step has 9 features.  
- y is (22950, 15): 22950 samples, each with a 15-dimensional one-hot vector (one station out of 15.


### Train/Test Split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,       # 25% test for example
    random_state=42
)

print(X_train.shape)  # e.g. (17212, 15, 9)
print(y_train.shape)  # e.g. (17212, 15)
print(X_test.shape)   # e.g. (5738, 15, 9)
print(y_test.shape)   # e.g. (5738, 15)

(17212, 15, 9)
(17212, 15)
(5738, 15, 9)
(5738, 15)


---

## Keras Model

### Hyperparameters

**Initial Hyperparameters**:
- Epochs = 10
- Batch size = 32
- n_hidden (filters for Conv1D or hidden units for RNN) = 64
- Activation (hidden layers) = "relu"
- Final layer activation = "softmax"
- Optimizer = "adam"
- Loss function = "categorical_crossentropy"

**Possible Changes**:
- Might increase `n_hidden` to 128 or 256 if accuracy is too low.
- Might increase `epochs` to 20 or 30 for better training.


In [29]:
# Basic hyperparameters
epochs = 10        # start small for testing
batch_size = 32
n_hidden = 64      # or 128, 256, etc.

timesteps = X_train.shape[1]   # 15
input_dim = X_train.shape[2]   # 9
n_classes = y_train.shape[1]   # 15

3b) Build the Model

### Why a CNN Model?

I chose a CNN (Convolutional Neural Network) because:

- CNNs can extract local spatial/temporal features efficiently.
- They can handle sequences where local patterns are important (here, we have 15 time steps).
- They tend to be faster to train than many RNN variants for this dataset size.
- They are a good baseline before exploring more-complex RNN or LSTM architectures.

In [32]:
model = Sequential()
# Example: CNN with kernel_size=2
model.add(Conv1D(filters=n_hidden, kernel_size=2, activation='relu',
                 input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(32, activation='relu'))          # a small Dense layer
model.add(Dense(n_classes, activation='softmax')) 
# Alternative final activations if multi-label or different constraints:
#  - 'sigmoid'  (common in multi-label classification)
#  - 'tanh'
#  - 'softmax'  (common in multi-class classification)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        14,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,079 (62.81 KB)

 Trainable params: 16,079 (62.81 KB)

 Non-trainable params: 0 (0.00 B)

3c) Compile and Train

In [34]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,  # or use X_val, y_val if you did a custom split
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)


Epoch 1/10
431/431 - 7s - 17ms/step - accuracy: 0.1124 - loss: 2001.6007 - val_accuracy: 0.0555 - val_loss: 2181.0244
Epoch 2/10
431/431 - 4s - 10ms/step - accuracy: 0.1014 - loss: 3855.5417 - val_accuracy: 0.4084 - val_loss: 6906.9023
Epoch 3/10
431/431 - 3s - 8ms/step - accuracy: 0.0943 - loss: 8551.9688 - val_accuracy: 0.2077 - val_loss: 16138.6416
Epoch 4/10
431/431 - 4s - 10ms/step - accuracy: 0.0975 - loss: 14309.9961 - val_accuracy: 0.0572 - val_loss: 15934.9756
Epoch 5/10
431/431 - 2s - 5ms/step - accuracy: 0.0971 - loss: 25610.1055 - val_accuracy: 0.0761 - val_loss: 36504.7891
Epoch 6/10
431/431 - 2s - 4ms/step - accuracy: 0.0996 - loss: 37361.5625 - val_accuracy: 0.0738 - val_loss: 29687.2930
Epoch 7/10
431/431 - 2s - 5ms/step - accuracy: 0.0963 - loss: 37704.4961 - val_accuracy: 0.0049 - val_loss: 49197.6523
Epoch 8/10
431/431 - 3s - 6ms/step - accuracy: 0.0955 - loss: 55445.4180 - val_accuracy: 0.6227 - val_loss: 65445.3164
Epoch 9/10
431/431 - 2s - 6ms/step - accuracy: 0.0

---

In [36]:
# final evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Final Test Accuracy:", test_accuracy)

Final Test Accuracy: 0.08313000947237015


## 4. Confusion Matrix

### Initial Confusion Matrix and Accuracy

Below is the confusion matrix and accuracy using the initial hyperparameters (`n_hidden=64`, `epochs=10`):

*Please see the printed matrix output below.* 

- Observations:
  - The model mostly predicts a few classes (BELGRADE, DUSSELDORF...).
  - Accuracy is around 19.01%.
  - It recognizes only 4 stations out of 15 (non-zero diagonal).


In [39]:
# PREDICT & GENERATE CONFUSION MATRIX

# Convert y_test one-hot => int
y_test_int = np.argmax(y_test, axis=1)

# Model predictions => int
y_pred_probas = model.predict(X_test)  # shape (n, 15)
y_pred_int = np.argmax(y_pred_probas, axis=1)

# Sklearn confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test_int, y_pred_int)
print("Confusion Matrix:\n", cm)

# Station names
stations = {
    0: 'BASEL', 1: 'BELGRADE', 2: 'BUDAPEST', 3: 'DEBILT',
    4: 'DUSSELDORF', 5: 'HEATHROW', 6: 'KASSEL', 7: 'LJUBLJANA',
    8: 'MAASTRICHT', 9: 'MADRID', 10: 'MUNCHENB', 11: 'OSLO',
    12: 'SONNBLICK', 13: 'STOCKHOLM', 14: 'VALENTIA'
}

y_test_names = pd.Series([stations[i] for i in y_test_int], name='True')
y_pred_names = pd.Series([stations[i] for i in y_pred_int], name='Pred')
ctab = pd.crosstab(y_test_names, y_pred_names)
print("\nCrosstab:\n", ctab)

# 3) MEASURE FINAL ACCURACY
accuracy = accuracy_score(y_test_int, y_pred_int)
print("\nFinal Accuracy (sklearn):", accuracy)

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Confusion Matrix:
 [[  24    0    0    0    0    0    0    0    0 3097    0    0   89  472
     0]
 [   0    0    0    0    0    0    0    0    0 1092    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0  214    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0   82    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0   29    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0   82    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0   11    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0   61    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    9    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0  453    0    0    1    4
     0]
 [   0    0    0    0    0    0    0    0    0    8    0    0    0    0
     0]
 [   0    0    0    0    0    0    0    0    0    5    0    

### Final Confusion Matrix and Accuracy

After increasing `n_hidden` to 128 and running for 20 epochs, the new accuracy i19.03XX%.  
Below is the confusion matrix and crosst not).


In [41]:
from tensorflow.keras.optimizers import Adam

# 1) Hyperparameters
#####################
n_hidden = 128   # number of filters/hidden units
epochs = 20      # you can try 10, 30, 50, etc.
batch_size = 32  # typical default

#####################
# 2) Shape Variables DONE
#####################
# X_train, X_test, y_train, y_test
# And they have shapes:
# X_train.shape => (n_train, 15, 9)
# y_train.shape => (n_train, 15)  [one-hot for 15 stations]
#
# timesteps = 15
# input_dim = 9
# n_classes = 15
timesteps = X_train.shape[1]   # 15
input_dim = X_train.shape[2]   # 9
n_classes = y_train.shape[1]   # 15

#####################
# 3) Build the Model
#####################
model = Sequential()

# Convolution layer
model.add(Conv1D(
    filters=n_hidden,
    kernel_size=2,
    activation='relu',
    input_shape=(timesteps, input_dim)
))

# Pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten away the remaining time dimension
model.add(Flatten())

# Optional dense layer
model.add(Dense(32, activation='relu'))

# Final dense layer => (None, 15) for multi-class classification
model.add(Dense(n_classes, activation='softmax'))

# Show model structure
model.summary()

#####################
# 4) Compile
#####################
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),  # or 'adam'
    metrics=['accuracy']
)

#####################
# 5) Train
#####################
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,  # or (X_val, y_val) if you have a separate val set
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

#####################
# 6) Evaluate on Test
#####################
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_accuracy)

#####################
# 7) Predict
#####################
y_pred_probas = model.predict(X_test)  # shape => (n_test, 15)
y_pred_int = np.argmax(y_pred_probas, axis=1)   # (n_test,)
y_test_int = np.argmax(y_test, axis=1)          # (n_test,)

#####################
# 8) Confusion Matrix
#####################
cm = confusion_matrix(y_test_int, y_pred_int)
print("\nConfusion Matrix:\n", cm)

acc = accuracy_score(y_test_int, y_pred_int)
print("\nAccuracy Score (sklearn):", acc)

#####################
# 9) Crosstab (Optional)
#####################
stations = {
    0: 'BASEL', 1: 'BELGRADE', 2: 'BUDAPEST', 3: 'DEBILT',
    4: 'DUSSELDORF', 5: 'HEATHROW', 6: 'KASSEL', 7: 'LJUBLJANA',
    8: 'MAASTRICHT', 9: 'MADRID', 10: 'MUNCHENB', 11: 'OSLO',
    12: 'SONNBLICK', 13: 'STOCKHOLM', 14: 'VALENTIA'
}

y_test_names = pd.Series([stations[i] for i in y_test_int], name='True')
y_pred_names = pd.Series([stations[i] for i in y_pred_int], name='Pred')

ctab = pd.crosstab(y_test_names, y_pred_names)
print("\nCrosstab:\n", ctab)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │        28,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,631 (123.56 KB)

 Trainable params: 31,631 (123.56 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
431/431 - 9s - 21ms/step - accuracy: 0.1081 - loss: 3021.5764 - val_accuracy: 0.0296 - val_loss: 3544.3030
Epoch 2/20
431/431 - 3s - 8ms/step - accuracy: 0.0996 - loss: 5647.3228 - val_accuracy: 0.1879 - val_loss: 10162.8525
Epoch 3/20
431/431 - 5s - 11ms/step - accuracy: 0.0975 - loss: 12494.8154 - val_accuracy: 0.0131 - val_loss: 16811.8555
Epoch 4/20
431/431 - 2s - 5ms/step - accuracy: 0.0999 - loss: 24791.8379 - val_accuracy: 0.0462 - val_loss: 25443.7852
Epoch 5/20
431/431 - 3s - 7ms/step - accuracy: 0.0986 - loss: 34311.6523 - val_accuracy: 0.3500 - val_loss: 47381.6562
Epoch 6/20
431/431 - 5s - 12ms/step - accuracy: 0.0994 - loss: 55280.9219 - val_accuracy: 0.0285 - val_loss: 59931.6680
Epoch 7/20
431/431 - 5s - 11ms/step - accuracy: 0.1002 - loss: 54278.4023 - val_accuracy: 0.0375 - val_loss: 46651.3438
Epoch 8/20
431/431 - 2s - 5ms/step - accuracy: 0.0988 - loss: 68448.7812 - val_accuracy: 0.0015 - val_loss: 48415.4609
Epoch 9/20
431/431 - 3s - 7ms/step - accuracy: 

In [42]:
# 1) Hyperparameters
#####################
n_hidden = 260   # number of filters/hidden units
epochs = 20      # you can try 10, 30, 50, etc.
batch_size = 32  # typical default

#####################
# 2) Shape Variables DONE
#####################
# X_train, X_test, y_train, y_test
# And they have shapes:
# X_train.shape => (n_train, 15, 9)
# y_train.shape => (n_train, 15)  [one-hot for 15 stations]
#
# timesteps = 15
# input_dim = 9
# n_classes = 15
timesteps = X_train.shape[1]   # 15
input_dim = X_train.shape[2]   # 9
n_classes = y_train.shape[1]   # 15

#####################
# 3) Build the Model
#####################
model = Sequential()

# Convolution layer
model.add(Conv1D(
    filters=n_hidden,
    kernel_size=2,
    activation='relu',
    input_shape=(timesteps, input_dim)
))

# Pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten away the remaining time dimension
model.add(Flatten())

# Optional dense layer
model.add(Dense(32, activation='relu'))

# Final dense layer => (None, 15) for multi-class classification
model.add(Dense(n_classes, activation='softmax'))

# Show model structure
model.summary()

#####################
# 4) Compile
#####################
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),  # or 'adam'
    metrics=['accuracy']
)

#####################
# 5) Train
#####################
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,  # or (X_val, y_val) if you have a separate val set
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

#####################
# 6) Evaluate on Test
#####################
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_accuracy)

#####################
# 7) Predict
#####################
y_pred_probas = model.predict(X_test)  # shape => (n_test, 15)
y_pred_int = np.argmax(y_pred_probas, axis=1)   # (n_test,)
y_test_int = np.argmax(y_test, axis=1)          # (n_test,)

#####################
# 8) Confusion Matrix
#####################
cm = confusion_matrix(y_test_int, y_pred_int)
print("\nConfusion Matrix:\n", cm)

acc = accuracy_score(y_test_int, y_pred_int)
print("\nAccuracy Score (sklearn):", acc)

#####################
# 9) Crosstab (Optional)
#####################
stations = {
    0: 'BASEL', 1: 'BELGRADE', 2: 'BUDAPEST', 3: 'DEBILT',
    4: 'DUSSELDORF', 5: 'HEATHROW', 6: 'KASSEL', 7: 'LJUBLJANA',
    8: 'MAASTRICHT', 9: 'MADRID', 10: 'MUNCHENB', 11: 'OSLO',
    12: 'SONNBLICK', 13: 'STOCKHOLM', 14: 'VALENTIA'
}

y_test_names = pd.Series([stations[i] for i in y_test_int], name='True')
y_pred_names = pd.Series([stations[i] for i in y_pred_int], name='Pred')

ctab = pd.crosstab(y_test_names, y_pred_names)
print("\nCrosstab:\n", ctab)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 14, 260)        │         4,940 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 260)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1820)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │        58,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 63,707 (248.86 KB)

 Trainable params: 63,707 (248.86 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
431/431 - 7s - 16ms/step - accuracy: 0.1126 - loss: 4825.5225 - val_accuracy: 0.0206 - val_loss: 10656.9355
Epoch 2/20
431/431 - 2s - 5ms/step - accuracy: 0.0919 - loss: 18426.4297 - val_accuracy: 0.0067 - val_loss: 29537.1289
Epoch 3/20
431/431 - 3s - 8ms/step - accuracy: 0.0957 - loss: 46732.1797 - val_accuracy: 0.0813 - val_loss: 81278.4922
Epoch 4/20
431/431 - 5s - 11ms/step - accuracy: 0.0927 - loss: 82950.2266 - val_accuracy: 0.0096 - val_loss: 97201.5078
Epoch 5/20
431/431 - 4s - 10ms/step - accuracy: 0.0922 - loss: 133874.6094 - val_accuracy: 0.1092 - val_loss: 141128.3438
Epoch 6/20
431/431 - 2s - 5ms/step - accuracy: 0.0965 - loss: 183918.8438 - val_accuracy: 0.0590 - val_loss: 162414.5938
Epoch 7/20
431/431 - 2s - 5ms/step - accuracy: 0.0970 - loss: 220809.7344 - val_accuracy: 0.0741 - val_loss: 169164.8125
Epoch 8/20
431/431 - 3s - 7ms/step - accuracy: 0.0938 - loss: 267912.3750 - val_accuracy: 8.7133e-04 - val_loss: 350363.6562
Epoch 9/20
431/431 - 3s - 6ms/step